# 02 — Feature Engineering

## Goals
- Build customer-level features from the cleaned session data produced by EDA.
- Save a deterministic customer feature table for segmentation.

## Inputs
- `config/features.yaml` (feature definitions and input/output paths).
- `artifacts/eda/latest/data/sessions_clean.parquet` from the EDA run.

## Outputs
- `artifacts/feature_engineering/customer_features.parquet` (customer-level features).


In [ ]:
from pathlib import Path

from traveltide.features.pipeline import run_features

outdir = Path("artifacts/feature_engineering")
outdir.mkdir(parents=True, exist_ok=True)

features_path = run_features(config_path="config/features.yaml", outdir=str(outdir))
features_path
